In [1]:
import ipynb.fs.full.get_news_headlines as gethl
import ipynb.fs.full.get_article_content as getart
import ipynb.fs.full.open_ai_models as openaimodel
import ipynb.fs.full.write_data as gswd
import ipynb.fs.full.read_data as gsrd
import requests
import pandas
import json
import os
import sys
import pprint as pp
import dotenv as env
env.load_dotenv('../keys.env')

#<script async src="https://cse.google.com/cse.js?cx=314b1cf0785af4af3">
#</script>
#<div class="gcse-search"></div>

#API Doc here: https://developers.google.com/custom-search/v1/reference/rest/v1/cse/list

True

In [2]:
SEARCH_API_KEY = os.environ.get('SEARCH_API_KEY')
cx_id = os.environ.get('SEARCH_CX_ID')
search_query = 'Sumeet+Bagadia' 
exact_terms = 'buy'
start_num = 1
headlines =[]
links = []
link_id = []
max_limit = 10    #max_limit to which Google will let us search

while start_num < max_limit:
    url = f'https://www.googleapis.com/customsearch/v1?key={SEARCH_API_KEY}&cx={cx_id}&q={search_query}&exact={exact_terms}&num=10&start={start_num}'
    response = requests.get(url)
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()
        #pp.pprint(data)
        # Print the search results (example)
        for item in data['items']:
            headlines.append(item['pagemap']['metatags'][0]['og:description']) #index called 'og:description  has a name of the link'
            links.append(item['link'])
            link_id.append(item['link'][-19:][:14])
        start_num += 10
    else:
        print(f'Error: {response.status_code}')
        break

search_headlines = pandas.DataFrame({
                                    #'uuid': uuids, 
                                    'title': headlines,     #Don't change these column names, they are important for merging later 
                                    'link': links,
                                    'link_id': link_id
                                    })
search_headlines.insert(0, 'site', 'Livemint')
search_headlines.drop_duplicates(subset=['link_id'], keep='first', inplace=True, ignore_index=True)

In [3]:
classified_hl = pandas.DataFrame(json.loads(gethl.classify_headlines(search_headlines['title'])))
classified_titles = search_headlines.join(classified_hl, lsuffix='_orig', rsuffix='_copy')   #make one df given with surety that matching is happening on indexes because Title matching used to give missed matches.. 
classified_titles = classified_titles[classified_titles.classification]
pp.pprint(classified_titles)

Executing headline classification for 10 headlines
       site                                         title_orig  \
0  Livemint  Sumeet Bagadia recommended buying these seven ...   
1  Livemint  Stocks to buy under  ₹100: Motherson Sumi Wiri...   
2  Livemint  Breakout stocks to buy or sell: Sumeet Bagadia...   
3  Livemint  Breakout stocks to buy or sell: Sumeet Bagadia...   
4  Livemint  Regarding stocks to buy under  ₹100, Sumeet Ba...   
5  Livemint  Buy or sell: Sumeet Bagadia recommends three s...   
6  Livemint  Buy or sell: Sumeet Bagadia recommends these t...   
7  Livemint  Buy or sell stocks: Sumeet Bagadia has recomme...   
8  Livemint  Breakout stocks to buy or sell: Sumeet Bagadia...   
9  Livemint  Breakout stocks to buy or sell: Sumeet Bagadia...   

                                                link         link_id  \
0  https://www.livemint.com/market/stock-market-n...  11743468724768   
1  https://www.livemint.com/market/stock-market-n...  11744519069538   
2  htt

In [3]:
model = openaimodel.OpenAIModels()
response = model.classify_headlines(search_headlines['title'])
pandas.DataFrame(json.loads(response)['output'])

,title,classification,explanation
0,Sumeet Bagadia recommended buying these seven ...,True,Contains 'recommended buying' phrase.
1,Stocks to buy under ₹100: Motherson Sumi Wiri...,True,Contains 'suggested buys' phrase.
2,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Contains 'recommends five shares to buy'.
3,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Contains 'recommends five shares to buy'.
4,"Regarding stocks to buy under ₹100, Sumeet Ba...",True,Contains 'recommended buying' phrase.
5,Buy or sell: Sumeet Bagadia recommends three s...,True,Contains 'recommends three stocks to buy'.
6,Buy or sell: Sumeet Bagadia recommends these t...,True,Contains 'recommends these three stocks to buy'.
7,Buy or sell stocks: Sumeet Bagadia has recomme...,True,Contains 'recommended three stocks to buy'.
8,Breakout stocks to buy or sell: Sumeet Bagadia...,True,Contains 'recommends five shares to buy'.
